In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class stylegan(object):
    def __init__(self, session, output_resolution=256):
        self.sess = session
        self.output_resolution = output_resolution
        self.num_style_blocks = 0
        
        self.conv_weights = []
        self.w_transforms = []
        
    def generator(self, W_in):
        with tf.variable_scope("generator") as vs:
            block_1 = tf.get_variable(
                "w1v",
                [512, 4, 4],
                initializer=tf.initializers.orthogonal
            )
            
    def styleBlock(self, V_in, latent_w, num_input_channels, num_output_channels, upsample=False):
        # V_in        --> [batch_size, height, width, num_input_channels]
        # latent_w    --> [batch_size, 512]
        # conv_weight --> [num_in_fm, num_out_fm, conv_dim]
        #    num_in_fm  = number of input feature maps
        #    num_out_fm = number of output feature maps
        #    conv_dim   = dimension of convolution
        self.num_style_blocks += 1
        
        A = tf.get_variable(
            "A_style" + str(self.num_style_blocks),
            [512, num_input_feature_maps],
            initializer=tf.initializers.orthogonal
        )
        
        conv_weight = tf.get_variable(
            "conv_w_style" + str(self.num_style_blocks),
            [3, 3, num_input_channels, num_output_channels],
            initialier=tf.initializers.orthogonal
        )
        
        # Affine transformation of latent space vector
        scale = tf.matmul(A, latent_w)
        
        # Scale input feature map acros input channels by the affine transformation
        # of the latent space input.
        V_in_scaled = tf.einsum("bi,bhwi->bhwi", scale, V_in)
        
        V_out = tf.nn.conv2d(V_in_scaled, conv_weight, padding="same")
        
        # This increases the number of weights by a factor of batch_size,
        # which is weird.
        modul_conv_weight = tf.einsum("bi,ijk->bijk", scale, conv_weight)
        sigma_j = tf.sqrt(tf.reduce_sum(tf.square(modul_conv_weight), axis=[1, 3]) + 1e-6)
        V_out_scaled = tf.nn.leaky_relu(
            tf.einsum("bhwj,bj->bhwj", V_in_scaled, sigma_j),
            alpha=0.2
        )

        return V_out_scaled
    
    def upsample(self, V_in):
        # Didn't test with the channel dimension yet... might be wrong.
        fm_size = tf.shape(V_in)
        batch_size, h, w, c = fm_size
        V_in_a = tf.concat([V_in, V_in,], axis=3)
        V_in_b = tf.reshape(V_in_a, [batch_size, 2*h, w, c])

        V_in_c = tf.transpose(V_in_b, perm=[0, 2, 1, 3])
        V_in_d = tf.concat([V_in_c, V_in_c], axis=3)
        V_in_e = tf.transpose(tf.reshape(e, [batch_size, 2*h, 2*w, c]), perm=[0, 2, 1, 3])

In [3]:
help(tf.reduce_sum)

Help on function reduce_sum_v1 in module tensorflow.python.ops.math_ops:

reduce_sum_v1(input_tensor, axis=None, keepdims=None, name=None, reduction_indices=None, keep_dims=None)
    Computes the sum of elements across dimensions of a tensor. (deprecated arguments)
    
    Instructions for updating:
    keep_dims is deprecated, use keepdims instead
    
    Reduces `input_tensor` along the dimensions given in `axis`.
    Unless `keepdims` is true, the rank of the tensor is reduced by 1 for each
    entry in `axis`. If `keepdims` is true, the reduced dimensions
    are retained with length 1.
    
    If `axis` is None, all dimensions are reduced, and a
    tensor with a single element is returned.
    
    For example:
    
    ```python
    x = tf.constant([[1, 1, 1], [1, 1, 1]])
    tf.reduce_sum(x)  # 6
    tf.reduce_sum(x, 0)  # [2, 2, 2]
    tf.reduce_sum(x, 1)  # [3, 3]
    tf.reduce_sum(x, 1, keepdims=True)  # [[3], [3]]
    tf.reduce_sum(x, [0, 1])  # 6
    ```
    
    Args:


In [4]:
help(tf.einsum)

Help on function einsum in module tensorflow.python.ops.special_math_ops:

einsum(equation, *inputs, **kwargs)
    A generalized contraction between tensors of arbitrary dimension.
    
    This function returns a tensor whose elements are defined by `equation`,
    which is written in a shorthand form inspired by the Einstein summation
    convention.  As an example, consider multiplying two matrices
    A and B to form a matrix C.  The elements of C are given by:
    
    ```
      C[i,k] = sum_j A[i,j] * B[j,k]
    ```
    
    The corresponding `equation` is:
    
    ```
      ij,jk->ik
    ```
    
    In general, the `equation` is obtained from the more familiar element-wise
    equation by
      1. removing variable names, brackets, and commas,
      2. replacing "*" with ",",
      3. dropping summation signs, and
      4. moving the output to the right, and replacing "=" with "->".
    
    Many common operations can be expressed in this way.  For example:
    
    ```python


In [5]:
help(tf.nn.conv2d)

Help on function conv2d in module tensorflow.python.ops.nn_ops:

conv2d(input, filter=None, strides=None, padding=None, use_cudnn_on_gpu=True, data_format='NHWC', dilations=[1, 1, 1, 1], name=None, filters=None)
    Computes a 2-D convolution given 4-D `input` and `filter` tensors.
    
    Given an input tensor of shape `[batch, in_height, in_width, in_channels]`
    and a filter / kernel tensor of shape
    `[filter_height, filter_width, in_channels, out_channels]`, this op
    performs the following:
    
    1. Flattens the filter to a 2-D matrix with shape
       `[filter_height * filter_width * in_channels, output_channels]`.
    2. Extracts image patches from the input tensor to form a *virtual*
       tensor of shape `[batch, out_height, out_width,
       filter_height * filter_width * in_channels]`.
    3. For each patch, right-multiplies the filter matrix and the image patch
       vector.
    
    In detail, with the default NHWC format,
    
        output[b, i, j, k] =
   

In [6]:
a = tf.constant(
    np.array(
        [
            [[1,2,3],
             [4,5,6],
             [7,8,9]
            ],
            [[-1,-2,-3],
             [-4,-5,-6],
             [-7,-8,-9]
            ],
            [[1.2,2.2,3.2],
             [4.2,5.2,6.2],
             [7.2,8.2,9.2]
            ],
        ]
    )
)

In [18]:
b = tf.concat([a, a,], axis=2)
c = tf.reshape(b, [3,6,3])

d = tf.transpose(c, perm=[0, 2, 1])
e = tf.concat([d, d], axis=2)
f = tf.transpose(tf.reshape(e, [3, 6, 6]), perm=[0, 2, 1])

In [19]:
sess = tf.Session()
sess.run(c)

array([[[ 1. ,  2. ,  3. ],
        [ 1. ,  2. ,  3. ],
        [ 4. ,  5. ,  6. ],
        [ 4. ,  5. ,  6. ],
        [ 7. ,  8. ,  9. ],
        [ 7. ,  8. ,  9. ]],

       [[-1. , -2. , -3. ],
        [-1. , -2. , -3. ],
        [-4. , -5. , -6. ],
        [-4. , -5. , -6. ],
        [-7. , -8. , -9. ],
        [-7. , -8. , -9. ]],

       [[ 1.2,  2.2,  3.2],
        [ 1.2,  2.2,  3.2],
        [ 4.2,  5.2,  6.2],
        [ 4.2,  5.2,  6.2],
        [ 7.2,  8.2,  9.2],
        [ 7.2,  8.2,  9.2]]])

In [20]:
sess.run(f)

array([[[ 1. ,  1. ,  2. ,  2. ,  3. ,  3. ],
        [ 1. ,  1. ,  2. ,  2. ,  3. ,  3. ],
        [ 4. ,  4. ,  5. ,  5. ,  6. ,  6. ],
        [ 4. ,  4. ,  5. ,  5. ,  6. ,  6. ],
        [ 7. ,  7. ,  8. ,  8. ,  9. ,  9. ],
        [ 7. ,  7. ,  8. ,  8. ,  9. ,  9. ]],

       [[-1. , -1. , -2. , -2. , -3. , -3. ],
        [-1. , -1. , -2. , -2. , -3. , -3. ],
        [-4. , -4. , -5. , -5. , -6. , -6. ],
        [-4. , -4. , -5. , -5. , -6. , -6. ],
        [-7. , -7. , -8. , -8. , -9. , -9. ],
        [-7. , -7. , -8. , -8. , -9. , -9. ]],

       [[ 1.2,  1.2,  2.2,  2.2,  3.2,  3.2],
        [ 1.2,  1.2,  2.2,  2.2,  3.2,  3.2],
        [ 4.2,  4.2,  5.2,  5.2,  6.2,  6.2],
        [ 4.2,  4.2,  5.2,  5.2,  6.2,  6.2],
        [ 7.2,  7.2,  8.2,  8.2,  9.2,  9.2],
        [ 7.2,  7.2,  8.2,  8.2,  9.2,  9.2]]])

In [ ]:
help(tf.reshape)

In [ ]:
help(tf.tile)

In [ ]:
help(tf.transpose)

In [ ]:
# Use matmuls for upsampling, see whiteboard